In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 3

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Molecular dynamics

We're going to use some principle from molecular dynamics (MD), but instead of simulating many particles we're going to keep it simple and only simulate a few.

## Verlet algorithm

Given initial data $r_i(0), v_i(0), a_i(0)$, using the Verlet algorithm we can calculate the first update in position using

\begin{equation}
    r_i(\Delta t) = r_i(0) + v_i(0) \Delta t + \frac{1}{2} a_i(0) \Delta t^2
    ,
\end{equation}

and subsequent changes in position using

\begin{equation}
    r_i(t + \Delta t) = 2 r_i(t) - r_i(t - \Delta t) + a_i(t) \Delta t^2
    ,
\end{equation}

where $r_i, v_i, a_i$ are understood to be vector quantities.

The first equation above should be self-explanatory. The second equation we can derive by making use of Taylor series:

\begin{equation}
    r_i(t \pm \Delta t)
        = r_i(t)
            \pm r_i^\prime(t) \Delta t
            + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
            \pm \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            + \mathcal{O}(\Delta t^4)
            .
\end{equation}

Thus

\begin{align}
    r_i(t + \Delta t)
        &= r_i(t)
            + r_i^\prime(t) \Delta t
            + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
            + \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            + \mathcal{O}(\Delta t^4) \\
        &= 2 r_i(t)
            + r_i^{\prime\prime}(t) \Delta t ^2
            - \left(
                r_i(t)
                - r_i^\prime(t) \Delta t
                + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
                - \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            \right)
            + \mathcal{O}(\Delta t^4) \\
        &= 2 r_i(t)
            + r_i^{\prime\prime}(t) \Delta t ^2
            - r_i(t - \Delta t)
            + \mathcal{O}(\Delta t^4) \\          
        &= 2 r_i(t)
            - r_i(t - \Delta t)
            + a_i(t) \Delta t ^2
            + \mathcal{O}(\Delta t^4)
        .
\end{align}

As we can see, this is good to $\mathcal{O}(\Delta t^4)$. (Note that the above derivation is equivalent to one using finite-difference methods, since those are based on Taylor series anyway.)

Let's write two functions that implement the Verlet algorithm functions.

In [ ]:
def verlet_1(r: np.ndarray, v: np.ndarray, a: np.ndarray, dt: float) -> np.ndarray:
    """Implements the first Verlet algorithm equation."""
    return r + dt * v + dt ** 2 / 2 * a

def verlet_2(r_1: np.ndarray, r_2: np.ndarray, a: np.ndarray, dt: float) -> np.ndarray:
    """Implements the second Verlet algorithm equation.
    
    Note that the input parameter r_1 specifies the position at time t,
    while the input parameter r_2 specifies the position at time t - dt.
    """
    return 2 * r_1 - r_2 + dt ** 2 * a

Let's test to make sure our functions do what they should do.

In [ ]:
# Set up initial data
init_pos = np.array([0, 3])
init_vel = np.array([1, 5])

# These are constant
accel = np.array([0, -9.8])
dt = 0.1

In [ ]:
# Test
num_iters = 20

t = 0
last_pos = init_pos
pos = init_pos

print_test = lambda r, t: print("t=%.2f\tx=%.2f\ty=%.2f" % (t, r[0], r[1]))

# 0th iter
print_test(pos, t)

# 1st iter
pos = verlet_1(pos, init_vel, accel, dt)
t += dt

print_test(pos, t)

# Remaining iters
for _ in range(num_iters - 1):
    pos, last_pos = (verlet_2(pos, last_pos, accel, dt), pos)
    t += dt
    
    print_test(pos, t)